---
layout: post
title: A first look at Mojo
categories: [mojo, coding]
excerpt: Taking a quick look at the new Mojo programming language. 
---

In [ ]:
from python import Python
from math import max
from utils.vector import DynamicVector
from algorithm.sort import sort

def read_file(file_name: String) -> DynamicVector[Int]:
    """There seems to be no native mojo way to read a file yet.

    There is an issue in the mojo github page that suggests the code below.
    https://github.com/modularml/mojo/issues/130

    I've tried to keep the Python code in this function, in order to keep the others
    as "pure" mojo
    """

    builtins = Python.import_module("builtins")
    in_file = builtins.open(file_name)

    file_contents = in_file.read()
    in_file.close()

    content_list = file_contents.split("\n")
    # There seems to be no way yet to go straight from python int to mojo
    # Int, so you have to go via float64: https://github.com/modularml/mojo/issues/657
    let list_len: Int = content_list.__len__().to_float64().to_int()

    item_list = DynamicVector[Int](list_len)
    
    for item in content_list:
        str_item = item.to_string()
        if str_item != "":
            item_list.push_back(atol(str_item))
        else:
            item_list.push_back(0)
        
    return item_list


fn part_one(calorie_list: DynamicVector[Int]) -> Int:

    var max_calories: Int = 0
    var this_calories: Int = 0


    var entry = 0
    for index in range(len(calorie_list)):
        entry = calorie_list.data[index]
        if entry != 0:
            this_calories += entry
        else:
            max_calories = max(this_calories, max_calories)
            this_calories = 0

    return max_calories

fn part_two(calorie_list: DynamicVector[Int]) -> Int:

    let k: Int = 3
    var max_k_calories: DynamicVector[Int] = DynamicVector[Int](k)
    for i in range(k):
        max_k_calories.push_back(0)
    var this_calories: Int = 0


    var entry = 0
    for index in range(len(calorie_list)):
        entry = calorie_list.data[index]
        if entry != 0:
            this_calories += entry
        else:
            if this_calories > max_k_calories[0]:
                max_k_calories[0] = this_calories
                sort(max_k_calories)


            this_calories = 0
            

    var max_calories: Int = 0
    for index in range(len(max_k_calories)):
        max_calories += max_k_calories.data[index]

    return max_calories


fn main() raises:
    let file_contents: DynamicVector[Int] = read_file(
        "/home/f/Workspace/advent_of_code_2022_mojo/input/input_1.txt"
    )

    let answer_1: Int = part_one(file_contents)
    print("The maximum calories carried by and elf is:", answer_1)

    let answer_2: Int = part_two(file_contents)
    print("The maximum calories carried by and elf is:", answer_2)
